# アソシエーション分析

- Apriori

#### Apriori

1. データを獲得する
         
      - 各種マスタ（ユーザー・アイテム）
      - 行動ログ（閲覧・お気に入り・カート投入・購買など）
      - 過去推薦ログ（レコメンドしたがクリックされなかった、のようなログ）
      - …　　
     
2. データを「リストのリスト」という形式に変換する

    - 以下のような購買データがある（評価値がない場合）
      - user1, itemA
      - user1, itemB
      - user2, itemA
      - user2, itemC
      - user3, itemD
      - user3, itemE  

    - 「リストのリスト」という形式に変換する：  
          
          [[itemA, itemB], [itemA, itemC], [itemD, itemE]]　　

3. Apriori 計算パッケージを利用する　

4. 指標結果
    - Support
    - Confidence
    - Lift　　

### データ読み込み

In [2]:
# -*- coding: utf-8 -*-
import pandas as pd

In [3]:
header = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('../data/ml-1m/ratings.dat', sep='::', names=header,engine='python')
# 実行時間を考慮して、10000行のデータを分析する
df = df.iloc[:10000]
df

,user_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


userとitemの数

In [4]:
# usersの種類
users = df['user_id'].unique()
# itemの種類
items = df['item_id'].unique()

users_num = users.shape[0]
items_num = items.shape[0]
print('usersの種類数： ' + str(users_num) + ' \nitemsの種類数： ' + str(items_num))

usersの種類数： 70 
itemsの種類数： 2159


### データを「リストのリスト」という形式に変換する

In [7]:
from functools import partial
def items_to_list(data, user_id):
    item_list = data.loc[data['user_id']==user_id]['item_id'].tolist()
    return item_list
func = partial(items_to_list, df)
item_list_list = list(map(func, users))
item_list_list

[[1193,
  661,
  914,
  3408,
  2355,
  1197,
  1287,
  2804,
  594,
  919,
  595,
  938,
  2398,
  2918,
  1035,
  2791,
  2687,
  2018,
  3105,
  2797,
  2321,
  720,
  1270,
  527,
  2340,
  48,
  1097,
  1721,
  1545,
  745,
  2294,
  3186,
  1566,
  588,
  1907,
  783,
  1836,
  1022,
  2762,
  150,
  1,
  1961,
  1962,
  2692,
  260,
  1028,
  1029,
  1207,
  2028,
  531,
  3114,
  608,
  1246],
 [1357,
  3068,
  1537,
  647,
  2194,
  648,
  2268,
  2628,
  1103,
  2916,
  3468,
  1210,
  1792,
  1687,
  1213,
  3578,
  2881,
  3030,
  1217,
  3105,
  434,
  2126,
  3107,
  3108,
  3035,
  1253,
  1610,
  292,
  2236,
  3071,
  902,
  368,
  1259,
  3147,
  1544,
  1293,
  1188,
  3255,
  3256,
  3257,
  110,
  2278,
  2490,
  1834,
  3471,
  589,
  1690,
  3654,
  2852,
  1945,
  982,
  1873,
  2858,
  1225,
  2028,
  515,
  442,
  2312,
  265,
  1408,
  1084,
  3699,
  480,
  1442,
  2067,
  1265,
  1370,
  1193,
  1801,
  1372,
  2353,
  3334,
  2427,
  590,
  1196,
  1552,
 

### Apriori 計算パッケージを利用する

In [11]:
from apyori import apriori
associations = apriori(item_list_list, min_support = 0.2, min_confidence = 1.0, min_lift = 4)
rule = list(associations)
rule

[RelationRecord(items=frozenset({1210, 260, 1374, 1356}), support=0.2, ordered_statistics=[OrderedStatistic(items_base=frozenset({1210, 260, 1356}), items_add=frozenset({1374}), confidence=1.0, lift=4.11764705882353)]),
 RelationRecord(items=frozenset({1210, 1196, 1374, 1356}), support=0.21428571428571427, ordered_statistics=[OrderedStatistic(items_base=frozenset({1210, 1196, 1356}), items_add=frozenset({1374}), confidence=1.0, lift=4.11764705882353)]),
 RelationRecord(items=frozenset({480, 260, 1356, 1210, 1374}), support=0.2, ordered_statistics=[OrderedStatistic(items_base=frozenset({480, 1210, 260, 1356}), items_add=frozenset({1374}), confidence=1.0, lift=4.11764705882353)]),
 RelationRecord(items=frozenset({260, 1196, 1356, 1210, 1374}), support=0.2, ordered_statistics=[OrderedStatistic(items_base=frozenset({1210, 1196, 260, 1356}), items_add=frozenset({1374}), confidence=1.0, lift=4.11764705882353)]),
 RelationRecord(items=frozenset({480, 1196, 1356, 1210, 1374}), support=0.214285

### 指標結果
   - Support
   - Confidence
   - Lift　

In [23]:
itemA = list(rule[0][2][0][0])
itemB = list(rule[0][2][0][1])
print(f'相関ルール： \n\t映画{itemA} を観た人が　映画{itemB} を観る')
print(f'指標結果：\n\tSupport: {rule[0][1]:.3}\n\tConfidence: {rule[0][2][0][2]:.3}\n\tLift　: {rule[0][2][0][3]:.3}')

itemC = list(rule[1][2][0][0])
itemD = list(rule[1][2][0][1])
print(f'\n相関ルール： \n\t映画{itemC} を観た人が　映画{itemD} を観る')
print(f'指標結果：\n\tSupport: {rule[1][1]:.3}\n\tConfidence: {rule[1][2][0][2]:.3}\n\tLift　: {rule[1][2][0][3]:.3}')

相関ルール： 
	映画[1210, 260, 1356] を観た人が　映画[1374] を観る
指標結果：
	Support: 0.2
	Confidence: 1.0
	Lift　: 4.12

相関ルール： 
	映画[1210, 1196, 1356] を観た人が　映画[1374] を観る
指標結果：
	Support: 0.214
	Confidence: 1.0
	Lift　: 4.12
